# Load GLEIF Data from public repository on AWS (s3://gleif/)
Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import pandas as pd
import trino
import io

In [3]:
# Use unsigned access to public S3 resource for GLEIF bucket
s3_resource = boto3.resource(
    service_name="s3",
    config=Config(signature_version=UNSIGNED),
)
bucket = s3_resource.Bucket("gleif")

In [4]:
# Retrieve latest LEI data set
LEI_data_sets = []
for obj in bucket.objects.filter(Prefix='data/csv/'):
    LEI_data_sets.append(obj.key)
    # print(obj.key)

LEI_data_sets[len(LEI_data_sets)-1]

'data/csv/rr/date=2021-10-25/time=08:00/20211025-0800-gleif-goldencopy-rr.csv.gz'

In [5]:
from botocore import UNSIGNED
from botocore.config import Config
client = boto3.client('s3', config=Config(signature_version=UNSIGNED)) #low-level functional API
obj = client.get_object(Bucket='gleif', Key=LEI_data_sets[len(LEI_data_sets)-1])
lei_data_set = pd.read_csv(obj['Body'], nrows=10, compression='gzip').convert_dtypes()

In [6]:
lei_data_set.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 54 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Relationship.StartNode.NodeID                 10 non-null     string
 1   Relationship.StartNode.NodeIDType             10 non-null     string
 2   Relationship.EndNode.NodeID                   10 non-null     string
 3   Relationship.EndNode.NodeIDType               10 non-null     string
 4   Relationship.RelationshipType                 10 non-null     string
 5   Relationship.RelationshipStatus               10 non-null     string
 6   Registration.InitialRegistrationDate          10 non-null     string
 7   Registration.LastUpdateDate                   10 non-null     string
 8   Registration.RegistrationStatus               10 non-null     string
 9   Registration.NextRenewalDate                  10 non-null     string
 10  Regis

In [7]:
lei_data_set

,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,...,Relationship.Qualifiers.2.QuantifierUnits,Relationship.Qualifiers.3.MeasurementMethod,Relationship.Qualifiers.3.QuantifierAmount,Relationship.Qualifiers.3.QuantifierUnits,Relationship.Qualifiers.4.MeasurementMethod,Relationship.Qualifiers.4.QuantifierAmount,Relationship.Qualifiers.4.QuantifierUnits,Relationship.Qualifiers.5.MeasurementMethod,Relationship.Qualifiers.5.QuantifierAmount,Relationship.Qualifiers.5.QuantifierUnits
0,010CMKZ3VON21WF2ZD45,LEI,ZZG38T0MDR3QY1ETUA76,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2012-10-01T11:09:00.000Z,2021-06-09T21:32:00.000Z,PUBLISHED,2022-06-03T13:04:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,010CMKZ3VON21WF2ZD45,LEI,3C7474T6CDKPR9K6YT90,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2012-10-01T11:09:00.000Z,2021-10-21T21:32:00.000Z,PUBLISHED,2022-06-03T13:04:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,010PWNH4K3BLIC3I7R03,LEI,549300COKYB5EGSU1838,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2012-11-26T15:01:00.000Z,2021-10-22T00:31:00.000Z,PUBLISHED,2022-10-21T15:08:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,010PWNH4K3BLIC3I7R03,LEI,549300B2Q47IR0CR5B54,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2012-11-26T15:01:00.000Z,2021-10-22T00:31:00.000Z,PUBLISHED,2022-10-21T15:08:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,01J4SO3XTWZF4PP38209,LEI,5493000HPQ4D2RZ79739,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2017-11-30T16:35:59.285Z,2020-12-01T17:12:02.857Z,PUBLISHED,2021-12-01T17:12:02.687Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,01J4SO3XTWZF4PP38209,LEI,5493000HPQ4D2RZ79739,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2017-11-30T16:35:59.285Z,2020-12-01T17:12:02.857Z,PUBLISHED,2021-12-01T17:12:02.687Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,01TRDHWDCL69YP41S025,LEI,NZKGWYDO5O20EZTLPF09,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2012-10-10T19:47:00.000Z,2021-04-16T15:36:00.000Z,PUBLISHED,2022-04-14T16:48:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,01TRDHWDCL69YP41S025,LEI,LORM1GNEU1DKEW527V90,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2012-10-10T19:47:00.000Z,2021-04-16T21:31:00.000Z,PUBLISHED,2022-04-14T16:48:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,020BQJXAXCZNLKIN7326,LEI,549300PFEWKNHRG25N08,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2012-11-28T14:55:00.000Z,2021-10-21T02:30:00.000Z,PUBLISHED,2022-01-06T21:32:00.000Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,0292001381F1R1IB5B85,LEI,029200388A7S7Z5I0H57,LEI,IS_DIRECTLY_CONSOLIDATED_BY,ACTIVE,2016-07-14T00:00:00Z,2021-07-12T09:30:19Z,PUBLISHED,2022-07-13T20:00:00Z,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
